In [ ]:
!pip install geopandas

: 

In [ ]:
import pandas as pd

iqvu = pd.read_csv("csvs/bd_iqvu_1994_2016.csv", sep=";", encoding="latin-1")

iqvu.rename(columns={
    'ï»¿SERIE': 'serie',
    'ANO': 'ano',
    'CODUP': 'cod_up',
    'NOMEUP': 'nome',
    'IQVU_3_Educacao': 'iqvu_educacao',
    'IQVU_5_Habitacao': 'iqvu_habitacao',
    'IQVU_6_Infra-estrutura_urbana': 'iqvu_infraestrutura',
    'IQVU_7_Meio_Ambiente': 'iqvu_meio_ambiente',
    'IQVU_8_Saude': 'iqvu_saude',
    'IQVU_9_Servicos_Urbanos': 'iqvu_servicos',
    'IQVU': 'iqvu'
}, inplace=True)

cols_num = [c for c in iqvu.columns if 'iqvu' in c.lower()]
for col in cols_num:
    iqvu[col] = iqvu[col].astype(str).str.replace(',', '.').astype(float)

# mantém apenas as colunas relevantes (as que de fato estão no CSV)
colunas_relevantes = [
    "ano",
    "cod_up",
    "nome",
    "iqvu",
    "iqvu_educacao",
    "iqvu_habitacao",
    "iqvu_infraestrutura",
    "iqvu_meio_ambiente",
    "iqvu_saude",
    "iqvu_servicos"
]

iqvu = iqvu[colunas_relevantes]

iqvu.head()



In [ ]:
risco_inundacao = pd.read_csv('csvs/20250707_area-prioritaria-_sbn.csv', sep=";", encoding='latin-1')

colunas_analise = [
    'AREA_M2',
    'EXP_ALAG', 'EXP_DESL', 'EXP_INUN',
    'VUL_ALAG', 'VUL_DESL', 'VUL_INUN',
    'CON_AMB', 'CON_ANT', 'CON_TOT',
    'SBN_ALAG', 'SBN_DESL', 'SBN_INUN'
]

risco_inundacao = risco_inundacao[colunas_analise]
risco_inundacao.rename(columns={
    'EXP_ALAG': 'exposicao_alagamento',
    'EXP_DESL': 'exposicao_deslizamento',
    'EXP_INUN': 'exposicao_inundacao',
    'VUL_ALAG': 'vulnerabilidade_alagamento',
    'VUL_DESL': 'vulnerabilidade_deslizamento',
    'VUL_INUN': 'vulnerabilidade_inundacao',
    'CON_AMB': 'condicoes_ambientais',
    'CON_ANT': 'condicoes_humanas',
    'CON_TOT': 'condicoes_totais',
    'SBN_ALAG': 'risco_alagamento',
    'SBN_DESL': 'risco_deslizamento',
    'SBN_INUN': 'risco_inundacao'
}, inplace=True)

for col in risco_inundacao.columns:
    risco_inundacao[col] = risco_inundacao[col].astype(str).str.replace(',', '.').astype(float)

risco_inundacao.describe().round(2)


In [ ]:
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt

# Ler CSV da SBN
area_prioritaria_sbn = gpd.read_file("csvs/20250707_area-prioritaria-_sbn.csv", delimiter=";")
area_prioritaria_sbn["geometry"] = area_prioritaria_sbn["GEOMETRIA"].apply(wkt.loads)
area_prioritaria_sbn = gpd.GeoDataFrame(area_prioritaria_sbn, geometry="geometry", crs="EPSG:31983")

# Corrigir campo numérico
area_prioritaria_sbn["SBN_INUN"] = (
    area_prioritaria_sbn["SBN_INUN"]
    .astype(str)
    .str.replace(",", ".")
    .str.extract("([0-9.]+)")[0]
    .astype(float)
)

#  Ler UPs
ups = gpd.read_file("csvs/20220601_unidade_planejamento.csv", delimiter=";")
ups["geometry"] = ups["GEOMETRIA"].apply(wkt.loads)
ups = gpd.GeoDataFrame(ups, geometry="geometry", crs="EPSG:31983")

# 4) Fazer o spatial join — associa cada área SBN à UP onde ela está
sbn_up = gpd.sjoin(
    area_prioritaria_sbn,
    ups[["NOME_UNID_PLANEJAMENTO", "geometry"]],
    how="left",
    predicate="intersects"
)

#  Agregar por UP  dissolve os polígonos da SBN dentro de cada UP
sbn_por_up = sbn_up.dissolve(
    by="NOME_UNID_PLANEJAMENTO",
    aggfunc={
        "SBN_INUN": "mean"  # média de risco por UP
    }
).reset_index()

#  Plotar
fig, ax = plt.subplots(figsize=(10, 10))

sbn_por_up.plot(
    ax=ax,
    column="SBN_INUN",
    cmap="Reds",
    linewidth=0.3,
    edgecolor="white",
    legend=True
)

ups.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.set_title("Média do Risco SBN por Unidade de Planejamento – BH", fontsize=14, pad=15)
ax.set_axis_off()
plt.tight_layout()
plt.show()


In [ ]:
# Lê o CSV dos bairros oficiais
bairros = pd.read_csv("csvs/20230502_bairro_oficial.csv", sep=",", encoding="latin-1")

# Converte a coluna GEOMETRIA em polígonos
bairros["geometry"] = bairros["GEOMETRIA"].apply(wkt.loads)

# Cria o GeoDataFrame
bairros = gpd.GeoDataFrame(bairros, geometry="geometry", crs="EPSG:31983")

# plot
fig, ax = plt.subplots(figsize=(10, 10))
bairros.plot(
    ax=ax,
    color="#a8dadc",
    edgecolor="gray",
    linewidth=0.5
)

ax.set_title("Bairros Oficiais de Belo Horizonte", fontsize=16, pad=20)
ax.set_axis_off()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))


area_prioritaria_sbn.plot(
    ax=ax,
    column="SBN_INUN",     # variável de cor
    cmap="Reds",           # tons de vermelho
    linewidth=0,           # sem borda
    alpha=0.8              # um pouco transparente
)


bairros.boundary.plot(
    ax=ax,
    color="gray",          # só contorno
    linewidth=0.5          # linha fininha
)

ax.set_title("Áreas SBN com Demarcação dos Bairros Oficiais de Belo Horizonte", fontsize=15, pad=20)
ax.set_axis_off()

plt.show()


In [ ]:
import unicodedata
import geopandas as gpd

# garante o mesmo sistema de coordenadas
area_prioritaria_sbn = area_prioritaria_sbn.to_crs(bairros.crs)

# interseção espacial: liga áreas SBN aos bairros
risco_com_bairro = gpd.sjoin(
    area_prioritaria_sbn,
    bairros[["NOME", "geometry"]],
    how="left",
    predicate="intersects"
)

# normaliza o texto sem tentar recodificar (evita erro UnicodeDecodeError)
risco_com_bairro["NOME"] = risco_com_bairro["NOME"].apply(
    lambda x: unicodedata.normalize("NFKC", x) if isinstance(x, str) else x
)

print("antes agregação de bairros:\n", risco_com_bairro.isna().sum(), end="\n\n")

# lista de colunas numéricas relevantes
cols_numericas = [
    "SBN_ALAG", "SBN_DESL", "SBN_INUN", "SBN_ONDA",
    "EXP_ALAG", "EXP_DESL", "EXP_INUN", "EXP_ONDA",
    "VUL_ALAG", "VUL_DESL", "VUL_INUN", "VUL_ONDA",
    "CON_AMB", "CON_ANT", "CON_TOT"
]

# garante que todas sejam numéricas (corrige vírgulas, converte pra float)
for c in cols_numericas:
    if c in risco_com_bairro.columns:
        risco_com_bairro[c] = (
            risco_com_bairro[c]
            .astype(str)
            .str.replace(",", ".")
            .str.extract(r"([-+]?\d*\.?\d+)")[0]
            .astype(float)
        )

# agrega estatísticas por bairro (médias e contagens)
agg_dict = {c: "mean" for c in cols_numericas if c in risco_com_bairro.columns}
agg_dict["geometry"] = "count"

resumo_bairros = (
    risco_com_bairro
    .groupby("NOME")
    .agg(agg_dict)
    .reset_index()
    .rename(columns={"geometry": "num_areas_risco"})
    .sort_values("num_areas_risco", ascending=False)
)

print("depois agregação bairros:\n",resumo_bairros.isna().sum())
print("Shape resumo_bairros: ", resumo_bairros.shape, end="\n\n")

print("Médias e contagens por bairro:")
display(resumo_bairros.head(10))


1 - AQUI COMECA A RESPONDER PERGUNTA 1 -
Esse código serve pra juntar os dados dos bairros com as Unidades de Planejamento (UPs).
Como alguns bairros ficam divididos entre mais de uma UP, ele usa a porcentagem de área de cada bairro dentro da UP pra fazer uma média ponderada.
Assim, se um bairro está 70% numa UP e 30% em outra, o valor dele é dividido exatamente nessa proporção.
No final, o resultado é uma tabela com os indicadores médios de cada UP, calculados de forma proporcional à área dos bairros que fazem parte dela.


In [ ]:

#  Ler a tabela de correspondência bairro → UP


bairros_por_up = pd.read_csv("csvs/bairros_por_up.csv", encoding="utf-8")

# Corrige espaços e nomes de colunas
bairros_por_up.columns = bairros_por_up.columns.str.strip()

# Confere se o nome da coluna está correto agora
# print(bairros_por_up.columns)
# deve mostrar: ['ID_BAC', 'NOME', 'NOME_UNID_PLANEJAMENTO', 'perc_bairro_na_up']


# Juntar o resumo dos bairros com a tabela de UPs

resumo_bairros_up = resumo_bairros.merge(
    bairros_por_up,
    on="NOME",   # nome do bairro
    how="left"
)


#  Calcular médias ponderadas por UP

resumo_bairros_up["peso"] = resumo_bairros_up["perc_bairro_na_up"] / 100

# identifica as colunas numéricas (todas as médias calculadas)
cols_numericas = [
    c for c in resumo_bairros_up.columns
    if c not in ["ID_BAC", "NOME", "NOME_UNID_PLANEJAMENTO", "perc_bairro_na_up", "peso"]
       and pd.api.types.is_numeric_dtype(resumo_bairros_up[c])
]

# cria colunas ponderadas
for c in cols_numericas:
    resumo_bairros_up[f"{c}_pond"] = resumo_bairros_up[c] * resumo_bairros_up["peso"]


# 4. Agregar por Unidade de Planejamento

agg_dict = {f"{c}_pond": "sum" for c in cols_numericas}
agg_dict["NOME"] = "count"

resumo_up = (
    resumo_bairros_up
    .groupby("NOME_UNID_PLANEJAMENTO")
    .agg(agg_dict)
    .rename(columns={"NOME": "num_bairros"})
    .reset_index()
)
print("Resumo_up NaNs\n", resumo_up.isna().sum(), end="\n") # NaNs de resumo_bairros some na agregação
print("Resumo_up shape: ", resumo_up.shape)

# remove o sufixo "_pond" dos nomes das colunas
resumo_up.columns = [col.replace("_pond", "") for col in resumo_up.columns]


# resultado

print("\nResumo final por Unidade de Planejamento:")
display(resumo_up.head(10))


Esse código descobre quanto de cada bairro está dentro de cada Unidade de Planejamento (UP).
Primeiro ele lê os mapas dos bairros e das UPs e transforma as informações de texto em formas geométricas que o Python entende.
Depois ele sobrepõe os dois mapas, cortando os bairros pelas fronteiras das UPs.
Com isso, ele mede a área de cada pedaço e calcula a porcentagem do bairro que fica em cada UP.
No fim, ele salva uma tabela mostrando, por exemplo, que um bairro está 70% em uma UP e 30% em outra.
No final ele gera um arquivo bairros_por_up.csv


In [ ]:
#  Ler os CSVs
bairros = pd.read_csv("csvs/20230502_bairro_oficial.csv")
ups = pd.read_csv("csvs/20220601_unidade_planejamento.csv")

#  Converter a coluna 'GEOMETRIA' de texto WKT para objetos geométricos
bairros["geometry"] = bairros["GEOMETRIA"].apply(wkt.loads)
ups["geometry"] = ups["GEOMETRIA"].apply(wkt.loads)

# Criar GeoDataFrames com o mesmo sistema de coordenadas (usa o mesmo CRS projetado)
bairros = gpd.GeoDataFrame(bairros, geometry="geometry", crs="EPSG:31983")
ups = gpd.GeoDataFrame(ups, geometry="geometry", crs="EPSG:31983")

# Corrigir eventuais geometrias inválidas
bairros["geometry"] = bairros.buffer(0)
ups["geometry"] = ups.buffer(0)

# Fazer a interseção espacial (cada pedaço do bairro que cai em uma UP)
intersec = gpd.overlay(bairros, ups, how="intersection")

# 6) Calcular a área de cada interseção (em m²)
intersec["area_intersect"] = intersec.geometry.area

# Calcular a proporção da área de cada bairro dentro de cada UP
area_bairro = intersec.groupby("ID_BAC")["area_intersect"].sum().rename("area_total_bairro")
intersec = intersec.merge(area_bairro, on="ID_BAC")
intersec["perc_bairro_na_up"] = (intersec["area_intersect"] / intersec["area_total_bairro"]) * 100

#  Se quiser só o par principal (UP dominante de cada bairro)
bairros_up = (
    intersec.sort_values("perc_bairro_na_up", ascending=False)
    .groupby("ID_BAC")
    .first()
    .reset_index()[["ID_BAC", "NOME", "NOME_UNID_PLANEJAMENTO", "perc_bairro_na_up"]]
)

#  Salvar resultado
bairros_up.to_csv("bairros_por_up.csv", index=False)

Esse código cria um mapa de correlação entre as variáveis numéricas do resumo por Unidade de Planejamento (UP).
Primeiro ele separa só as colunas com números, depois calcula a correlação entre elas , ou seja o quanto uma variável está relacionada com a outra.
Por fim, ele desenha um heatmap (mapa de calor) colorido: tons de vermelho mostram correlação positiva (as variáveis aumentam juntas) e tons de azul mostram correlação negativa (quando uma aumenta, a outra diminui).
É uma forma visual de entender quais indicadores têm comportamento parecido ou contrário dentro das UPs.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Selecionar apenas colunas numéricas
variaveis_numericas_up = resumo_up.select_dtypes(include=['float64', 'int64']).columns

# Remover algumas colunas
variaveis_filtradas = [
    col for col in variaveis_numericas_up
    if not col.startswith("SBN") and col not in ["num_bairros", "num_areas_risco"]
]

# Criar DataFrame apenas com as colunas numéricas filtradas
df_corr_up = resumo_up[variaveis_filtradas]

# Matriz de correlação
corr_matrix_up = df_corr_up.corr()

# Plotar o heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix_up,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    square=True,
    cbar_kws={"shrink": 0.8}
)

plt.title("Correlação entre variáveis médias por Unidade de Planejamento", fontsize=14, pad=20)
plt.tight_layout()

# Exportar imagem
plt.savefig("imgs/mapa_calor_up.png", dpi=300, bbox_inches='tight')

plt.show()


Esse código junta os dados do IQVU mais recente com o resumo das Unidades de Planejamento (UPs).
Primeiro ele escolhe, pra cada UP, o último ano disponível do IQVU , ou seja, a versão mais atual do índice.
Depois ele padroniza os nomes, deixando tudo em maiúsculas e sem espaços extras, pra evitar erro na hora de juntar.
Por fim, ele faz a junção das duas tabelas: o resumo das UPs de um lado e os valores do IQVU do outro.
O resultado é uma tabela completa, com as informações atuais de qualidade de vida associadas a cada Unidade de Planejamento.

In [ ]:
# Seleciona o último ano de IQVU por UP
iqvu_recente = (
    iqvu.loc[iqvu.groupby("nome")["ano"].idxmax()]
    .reset_index(drop=True)
)

print("iqvu NaNs:\n", iqvu_recente.isna().sum())
print("iqvu shape: ", iqvu_recente.shape, end="\n\n")

# Padroniza nomes
resumo_up["NOME_UNID_PLANEJAMENTO"] = resumo_up["NOME_UNID_PLANEJAMENTO"].str.upper().str.strip()
iqvu_recente["nome"] = iqvu_recente["nome"].str.upper().str.strip()

# Faz o merge
dados_up = resumo_up.merge(
    iqvu_recente,
    left_on="NOME_UNID_PLANEJAMENTO",
    right_on="nome",
    how="inner"
)

print("dados_up NaNs: \n", dados_up.isna().sum())
print(dados_up.head())
print(dados_up.shape)


Esse código mede como cada parte do IQVU se relaciona com o risco socioambiental.
Ele começa listando os subíndices do IQVU (educação, habitação, infraestrutura, etc.) e depois calcula a correlação de cada um deles com o indicador de risco médio das UPs.
O resultado mostra quais dimensões da qualidade de vida estão mais ligadas ao aumento ou à redução do risco.
Por fim, ele desenha um gráfico de barras pra visualizar essas correlações: barras negativas indicam que, quando a qualidade naquela área piora, o risco tende a aumentar.

In [ ]:


# Lista de subíndices do IQVU
subindices = [
    "iqvu_educacao",
    "iqvu_habitacao",
    "iqvu_infraestrutura",
    "iqvu_meio_ambiente",
    "iqvu_saude",
    "iqvu_servicos"
]

# Calcula correlação de cada subíndice com o risco socioambiental médio
corrs = {sub: dados_up[sub].corr(dados_up["CON_TOT"]) for sub in subindices}

# Transforma em DataFrame para visualizar melhor
corr_df = (
    pd.DataFrame.from_dict(corrs, orient="index", columns=["correlação"])
    .sort_values("correlação", ascending=True)
)

print(corr_df)

# Gráfico de barras
plt.figure(figsize=(8,5))
sns.barplot(x=corr_df.index, y=corr_df["correlação"], palette="mako")
plt.title("Correlação entre subíndices do IQVU e risco socioambiental médio", fontsize=13, pad=15)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Correlação (r)")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#
dados_2016 = dados_up[dados_up["ano"] == 2016].copy()


subindices = [
    "iqvu_educacao",
    "iqvu_habitacao",
    "iqvu_infraestrutura",
    "iqvu_meio_ambiente",
    "iqvu_saude",
    "iqvu_servicos"
]

# Colunas que devem ser excluídas do cálculo
colunas_excluir = ["ano", "cod_up", "num_bairros", "num_areas_risco"]

# Seleciona as colunas numéricas, excluindo as que começam com "SBN" ou estão na lista de exclusão
variaveis_sem_sbn = [
    col for col in dados_2016.select_dtypes(include=['float64', 'int64']).columns
    if not col.startswith("SBN") and col not in colunas_excluir
]

# Mantém só as variáveis numéricas que não são IQVU
variaveis_out = [v for v in variaveis_sem_sbn if v not in subindices]

# Cria matriz de correlação
corr_iqvu = dados_2016[subindices + variaveis_out].corr()
corr_iqvu = corr_iqvu.loc[subindices, variaveis_out]

# Plotar o heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(
    corr_iqvu,
    annot=True,
    cmap="coolwarm",
    fmt=".2f",
    cbar_kws={"shrink": 0.8}
)
plt.title("Correlação entre subíndices do IQVU (2016) e variáveis socioambientais", fontsize=13, pad=20)
plt.xlabel("Variáveis socioambientais")
plt.ylabel("Subíndices do IQVU")
plt.tight_layout()

plt.savefig("imgs/corr_iqvu_2016.png", dpi=300, bbox_inches='tight')
plt.show()


Esse código faz um teste de correlação de Pearson pra ver o quanto cada tipo de risco (como alagamento, deslizamento ou inundação) está ligado ao risco total das áreas.
Ele calcula, pra cada variável, o valor do coeficiente r, que mostra se a relação é forte e se é positiva ou negativa, e o valor-p, que indica se essa relação é estatisticamente confiável.
No fim, ele organiza tudo em uma tabela com os resultados e salva como uma imagem, pra visualizar de forma simples quais tipos de risco mais influenciam no risco total.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Filtra apenas o ano de 2016
dados_2016 = dados_up[dados_up["ano"] == 2016].copy()

# Subíndices do IQVU
subindices = [
    "iqvu_educacao",
    "iqvu_habitacao",
    "iqvu_infraestrutura",
    "iqvu_meio_ambiente",
    "iqvu_saude",
    "iqvu_servicos"
]

# Variáveis de risco da base SBN
variaveis_risco = [
    'EXP_ALAG', 'EXP_DESL', 'EXP_INUN', 'EXP_ONDA',
    'VUL_ALAG', 'VUL_DESL', 'VUL_INUN', 'VUL_ONDA',
    'CON_AMB', 'CON_ANT'
]


# 1️⃣ Teste de Pearson – variáveis SBN

resultados_sbn = []
for var in variaveis_risco:
    if var in dados_2016.columns and 'CON_TOT' in dados_2016.columns:
        r, p = pearsonr(dados_2016[var], dados_2016['CON_TOT'])
        resultados_sbn.append({
            "Variável": var,
            "Coeficiente r": round(r, 3),
            "Valor-p": f"{p:.2e}"
        })

pearson_sbn = pd.DataFrame(resultados_sbn).sort_values("Coeficiente r", ascending=False)

# Exibe e salva a tabela como imagem
fig, ax = plt.subplots(figsize=(7, len(pearson_sbn)*0.45 + 1))
ax.axis('off')

tabela = ax.table(
    cellText=pearson_sbn.values,
    colLabels=pearson_sbn.columns,
    cellLoc='center',
    loc='center'
)
tabela.auto_set_font_size(False)
tabela.set_fontsize(9)
tabela.auto_set_column_width(col=list(range(len(pearson_sbn.columns))))
for (row, col), cell in tabela.get_celld().items():
    if row == 0:
        cell.set_facecolor('#1a5276')
        cell.set_text_props(color='white', weight='bold')

plt.title("Teste de Correlação de Pearson – Variáveis de Risco (SBN)", fontsize=12, pad=15)
plt.savefig("imgs/teste_pearson_sbn.png", dpi=300, bbox_inches="tight")
plt.show()


# Teste de Pearson – IQVU x CON_TOT

resultados_iqvu = []
for sub in subindices:
    if sub in dados_2016.columns and 'CON_TOT' in dados_2016.columns:
        r, p = pearsonr(dados_2016[sub], dados_2016['CON_TOT'])
        resultados_iqvu.append({
            "Subíndice IQVU": sub,
            "Coeficiente r": round(r, 3),
            "Valor-p": f"{p:.2e}"
        })

pearson_iqvu = pd.DataFrame(resultados_iqvu).sort_values("Coeficiente r", ascending=False)

fig, ax = plt.subplots(figsize=(7, len(pearson_iqvu)*0.45 + 1))
ax.axis('off')

tabela2 = ax.table(
    cellText=pearson_iqvu.values,
    colLabels=pearson_iqvu.columns,
    cellLoc='center',
    loc='center'
)
tabela2.auto_set_font_size(False)
tabela2.set_fontsize(9)
tabela2.auto_set_column_width(col=list(range(len(pearson_iqvu.columns))))
for (row, col), cell in tabela2.get_celld().items():
    if row == 0:
        cell.set_facecolor('#117a65')
        cell.set_text_props(color='white', weight='bold')

plt.title("Teste de Correlação de Pearson – IQVU (2016) x Índice Total de Risco", fontsize=12, pad=15)
plt.savefig("imgs/teste_pearson_iqvu.png", dpi=300, bbox_inches="tight")
plt.show()


### Pergunta 1

####É possível construir um modelo preditivo que identifique bairros com potencial de se tornarem áreas de alto risco socioambiental nos próximos anos? Seria respondido mediante os atributos que designam o risco de ocorrer inundação, alagamento, deslizamento em uma área e desses valores é possível definir se uma área sofrerá constantemente com causas geológicas no futuro.

Antes de analisar as relações entre risco socioambiental e condições urbanas, foi preciso padronizar os dados no mesmo nível territorial. Enquanto o IQVU é calculado por Unidade de Planejamento (UP), outros indicadores estavam por bairro. Para compará-los, os dados de bairro foram convertidos em médias ponderadas por UP, usando como peso a proporção de área que cada bairro ocupa dentro da unidade. Isso garantiu que regiões maiores tivessem mais influência nos resultados e evitou distorções.

Com os dados consolidados por UP, foi feita uma análise exploratória para identificar como os indicadores socioambientais se relacionam entre si. Foram mantidas apenas as variáveis numéricas e retiradas as que indicavam áreas prioritárias de Soluções Baseadas na Natureza (SBN), pois representam propostas de intervenção e não riscos diretos.

![](imgs/correlacaoEntreVariaveisMedias.png)

A partir disso, foi construída uma matriz de correlação e um mapa de calor para visualizar as relações entre os indicadores. As correlações positivas e fortes mostraram que diferentes tipos de risco, como alagamento, deslizamento ou ondas de calor, tendem a ocorrer nas mesmas regiões. Isso indica que as vulnerabilidades urbanas são interligadas e refletem condições estruturais comuns, como adensamento, falta de drenagem e escassez de áreas verdes.

![](imgs/correlacaoSubindVariaveisMedias.png)

Na sequência, os dados socioambientais foram comparados aos subíndices do IQVU de 2016 (educação, habitação, infraestrutura, meio ambiente, saúde e serviços). As correlações, em geral, foram baixas ou negativas: áreas com piores condições urbanas também são as mais expostas a riscos e vulnerabilidades. Educação mostrou correlação positiva moderada, sugerindo que o acesso à educação tende a reduzir a exposição e a vulnerabilidade. Já habitação, infraestrutura e saúde apresentaram correlações fracas e negativas, reforçando a relação entre precariedade urbana e risco.

Esses resultados confirmam que as desigualdades urbanas e ambientais em Belo Horizonte se sobrepõem: as regiões com menor qualidade de vida são também as mais vulneráveis aos impactos socioambientais.

Foram feitos testes de correlação de Pearson para verificar se existe relação entre os diferentes tipos de risco socioambiental da base SBN e também entre esses riscos e os subíndices do IQVU de 2016. O teste mostra se duas variáveis andam juntas — por exemplo, se quando uma aumenta a outra também tende a aumentar.

A hipótese nula (H₀) diz que não existe relação significativa entre as variáveis.
A hipótese alternativa (H₁) diz que existe uma relação significativa.
Foi usado um nível de confiança de 95%.

Nos resultados da base SBN, quase todas as variáveis tiveram correlações positivas e altas com o índice total de risco, e valores-p muito baixos (menores que 0,001). Isso indica que os diferentes tipos de risco aparecem nas mesmas regiões, ou seja, onde há um problema, normalmente existem outros também.

Já na comparação com o IQVU, as correlações foram baixas e, em alguns casos, negativas. Isso mostra que as áreas com piores condições urbanas tendem a ter maiores riscos ambientais, mas essa relação não é forte nem uniforme.

### Pergunta 2

#### Quais são as variáveis urbanas e sociais com maior poder de influência na determinação da vulnerabilidade? Um estudo será feito em torno das variáveis de qualidade de vida em relação ao risco naquela área, para dizer qual contribui mais para a ocorrência de vulnerabilidade.

![](imgs/correlacaoEntreSubindices.jpeg)


A pergunta dois foi respondida com base na análise feita na pergunta um e tendo em vista esse gráfico que ilustra a correlação entre subíndices do IQVU e o risco socioambiental médio

Dessa forma, a partir das correlações calculadas, foi possível identificar quais variáveis urbanas exercem maior influência sobre a vulnerabilidade socioambiental. Entre os subíndices do IQVU, educação e meio ambiente foram os que apresentaram as relações mais expressivas com o risco médio. O subíndice de educação apresentou correlação positiva moderada, indicando que regiões com melhores condições educacionais tendem a ter menor exposição a riscos. Isso sugere que o acesso à educação atua como um fator de proteção, ampliando a capacidade de adaptação das populações e contribuindo para práticas mais conscientes de prevenção e gestão urbana.

Em contrapartida, o subíndice de meio ambiente apresentou a correlação negativa mais acentuada, evidenciando que áreas com piores condições ambientais são também as mais suscetíveis a impactos, como alagamentos, deslizamentos e ondas de calor. Essa relação reforça que a degradação ambiental não apenas expressa desigualdades territoriais, mas também intensifica a vulnerabilidade socioambiental.

Os subíndices de habitação, infraestrutura e saúde mostraram correlações negativas mais discretas, o que indica uma influência indireta sobre os níveis de vulnerabilidade, enquanto serviços apresentou correlação próxima de zero, sugerindo uma relação menos significativa.

De forma geral, esses resultados confirmam que a vulnerabilidade socioambiental em Belo Horizonte está associada a um conjunto de fatores interligados, em que a qualidade ambiental e o acesso à educação se destacam como dimensões centrais. A partir dessa análise, essas variáveis podem ser priorizadas em estudos preditivos, contribuindo para a identificação de áreas com maior potencial de se tornarem zonas de risco e para o direcionamento de políticas públicas mais eficazes de mitigação e adaptação.

### Pergunta 3

#### Os dados conseguem responder se as condicionantes antrópicas influenciam diretamente em um maior risco socioambiental ?

Hipótese Nula (H₀): As condicionantes antrópicas não influenciam em um maior risco socioambiental, isto é, não deve haver uma correlação entre os valores de condicionantes antrópicas e os valores de exposição, vulnerabilidade socioambiental e de áreas prioritárias de implementação SBN.

Hipótese alternativa (H₁): As condicionantes antrópicas influenciam o risco socioambiental, isto é, há correlação significativa entre os valores de condicionantes antrópicas e as variáveis de exposição, vulnerabilidade e SBN.

Será realizada a análise diretamente sobre a base de dados original, sem qualquer tipo de agregação ou junção com outras bases externas, de modo que a variável condicionantes antrópicas seja avaliada no mesmo contexto das suas variáveis de risco correspondentes. Foi investigado a quantidade de NaNs na base e removemos as colunas que tinham um número considerável de valores ausentes (removemos as que tinham aprox. mais de 20% de NaNs na feature), para evitar lacunas na futura análise se tal variável é correlacionado ou não a variável dependente. Assim, obtemos um dataset consistente e adequado para aplicar o teste de hipótese.

A técnica aplicada para testar essa hipótese foi o teste de correlação de Pearson (r), que mede o grau e a direção da relação linear entre duas variáveis quantitativas. Essa técnica fornece o valor r, que é a intensidade e direção da correlação (varia de –1 a +1) e p-value, que é o nível de significância estatística (probabilidade de observar esse valor r se H₀ fosse verdadeira).

Critérios de decisão:
 - Se valor-p < 0,05, rejeita-se H₀ (há correlação significativa).
 - Se valor-p ≥ 0,05, não se rejeita H₀ (não há correlação significativa).

In [ ]:
df = pd.read_csv('csvs/20250707_area-prioritaria-_sbn.csv', sep=";", encoding='latin-1')

print(df.isna().sum())
df_copy = df.drop(columns={'VUL_DESL', 'VUL_INUN', 'AC_DESL', 'AC_INUN', 'SBN_DESL', 'SBN_INUN', 'VUL_ALAG'})
print("NaNs após remoção: ", sum(df_copy.isna().sum()))

variaveis_risco = [
    'SBN_ALAG', 'SBN_ONDA', 'EXP_ALAG',
    'EXP_DESL', 'EXP_INUN', 'EXP_ONDA',
    'VUL_ONDA'
]

resultados = []
for var in variaveis_risco:
    r, p = pearsonr(df[var], df['CON_ANT'])
    resultados.append({
        "Variável": var,
        "Coeficiente r": round(r, 3),
        "Valor-p": f"{p:.2e}"
    })

pearson_df = pd.DataFrame(resultados).sort_values("Coeficiente r", ascending=False)

fig, ax = plt.subplots(figsize=(7, len(pearson_df)*0.45 + 1))
ax.axis('off')

tabela = ax.table(
    cellText=pearson_df.values,
    colLabels=pearson_df.columns,
    cellLoc='center',
    loc='center'
)


tabela.auto_set_font_size(False)
tabela.set_fontsize(9)
tabela.auto_set_column_width(col=list(range(len(pearson_df.columns))))


for (row, col), cell in tabela.get_celld().items():
    if row == 0:
        cell.set_facecolor('#1a5276')
        cell.set_text_props(color='white', weight='bold')

plt.savefig("imgs/teste_pearson_Q3.png", dpi=300, bbox_inches="tight")
plt.show()

#### Análise dos resultados

- Para as variáveis SBN_ONDA, SBN_ALAG, VUL_ONDA, EXP_ALAG e EXP_ONDA, os valores-p são extremamente baixos. Assim, rejeita-se a H₀, pois há correlação estatisticamente significativa entre as condicionantes antrópicas e esses indicadores de risco.

- Para EXP_INUN e EXP_DESL, os valores-p > 0,05 não se rejeita H₀, pois não há evidência de correlação significativa.
➜ Esses tipos de risco parecem não estar diretamente associados às pressões antrópicas.

Lembrando que a direção positiva de r indica que quanto maiores as condicionantes antrópicas, maior tende a ser o risco socioambiental (vulnerabilidade, exposição ou necessidade de SBN).

Dessa forma, com base nessa análise e no teste de correlação de Pearson, foi possível identificar associações estatisticamente significativas entre as condicionantes antrópicas e a maioria dos indicadores de risco socioambiental analisados. As correlações mais fortes ocorreram para as variáveis relacionadas às Soluções Baseadas na Natureza (SBN) e à vulnerabilidade ao perigo de ondas, indicando que áreas com maior pressão antrópica também concentram maior necessidade de intervenção, pois há suscetibilidade a impactos ambientais. Assim, rejeita-se parcialmente a hipótese nula, uma vez que há evidência de que as condicionantes antrópicas influenciam diretamente o aumento do risco socioambiental em determinados contextos.

### Pergunta 4
#### De que forma alterações simuladas na infraestrutura urbana impactam o mapa de risco da cidade? Seria feito modificações nos valores de riscos e se isso geraria na diminuição da vulnerabilidade, correlacionando com as possíveis políticas públicas voltada para mitigação desses riscos.


Hipótese Nula (H₀): Alterações simuladas na infraestrutura urbana não impactam na vulnerabilidade socioambiental.

Hipótese Alternativa (H¹): Alterações simuladas na infraestrutura urbana impactam na vulnerabilidade socioambiental.

A análise exploratório e o teste de hipótese seguem a mesma linha de raciocínio dita na questão 3, com a diferença que estamos focando agora na relação entre as pressões antrópicas e a vulnerabilidade/exposição

In [ ]:
variaveis_risco = [
    'EXP_ALAG', 'EXP_DESL', 'EXP_INUN',
    'EXP_ONDA','VUL_ONDA'
]

resultados = []
for var in variaveis_risco:
    r, p = pearsonr(df[var], df['CON_ANT'])
    resultados.append({
        "Variável": var,
        "Coeficiente r": round(r, 3),
        "Valor-p": f"{p:.2e}"
    })

pearson_df = pd.DataFrame(resultados).sort_values("Coeficiente r", ascending=False)

fig, ax = plt.subplots(figsize=(7, len(pearson_df)*0.45 + 1))
ax.axis('off')

tabela = ax.table(
    cellText=pearson_df.values,
    colLabels=pearson_df.columns,
    cellLoc='center',
    loc='center'
)


tabela.auto_set_font_size(False)
tabela.set_fontsize(9)
tabela.auto_set_column_width(col=list(range(len(pearson_df.columns))))


for (row, col), cell in tabela.get_celld().items():
    if row == 0:
        cell.set_facecolor('#1a5276')
        cell.set_text_props(color='white', weight='bold')

A imagem diz como a variável de condicionantes antrópicas (CONT_ANT) se correlaciona com as demais variáveis que são indicadores de vulnerabildidade e exposição, dessa forma, futuramente, poderemos simular valores em CONT_ANT com o intuito de averiguar se isso resulta em alguma mudança nessas features de indicadores.

Com isso em vista, vemos que para algumas variáveis há estatística significativa (VUL_ONDA, EXP_ALAG) para rejeitar a hipótese e outras que não da para rejeitar a hipótese (EXP_INUN, EXP_DESL). Entretanto, os valores r das que são estatísticamente significativa sobressaem das outras, assim, podemos parcialmente rejeitar a hipótese nula.

#### Simulação do impacto de melhorias de infraestrutura sobre os riscos



In [ ]:
from sklearn.linear_model import LinearRegression

# --- 1. configuração ---
CSV_PATH = "csvs/20250707_area-prioritaria-_sbn.csv"
FEATURE = 'CON_ANT'
TARGETS = ['VUL_ONDA', 'EXP_ALAG'] # alvos da simulação
FATOR_REDUCAO = 0.5 # melhoria de 50%

print(f"--- Simulação: Impacto de '{FEATURE}' (Redução de 50%) sobre {TARGETS} ---")

# --- 2. carregar dados ---
df = pd.read_csv(CSV_PATH, delimiter=';')

resultados = []

# --- 3. modelar e simular para cada alvo ---
for target in TARGETS:

    # preparar dados (remover NaNs do par)
    df_temp = df[[FEATURE, target]].dropna()

    if not df_temp.empty:
        X_original = df_temp[[FEATURE]]
        y_original = df_temp[target]

        # treinar modelo: Risco = B0 + B1 * CON_ANT
        model = LinearRegression()
        model.fit(X_original, y_original)

        # calcular média original
        media_risco_original = y_original.mean()

        # criar dados simulados (CON_ANT * 0.5)
        X_simulado = X_original * FATOR_REDUCAO

        # prever novo risco e calcular nova média
        y_pred_simulado = model.predict(X_simulado)
        media_risco_simulado = y_pred_simulado.mean()

        # calcular impacto
        mudanca_perc = ((media_risco_simulado - media_risco_original) / abs(media_risco_original)) * 100

        resultados.append({
            'Variavel_Alvo': target,
            'Media_Original': media_risco_original,
            'Media_Simulada': media_risco_simulado,
            'Mudanca_Percentual (%)': mudanca_perc
        })

# --- 4. apresentar resultados ---
df_resultados = pd.DataFrame(resultados)
print(df_resultados.to_markdown(index=False, floatfmt=".4f"))

A simulação baseada em Regressão Linear confirma que as condicionantes antrópicas (CON_ANT) têm um impacto direto e substancial sobre os riscos socioambientais. Ao simular uma melhoria de 50% na infraestrutura (reduzindo CON_ANT pela metade), os resultados mostram uma diminuição significativa nas variáveis alvo. Especificamente, a Vulnerabilidade à Onda (VUL_ONDA) apresentou uma redução de 44.68%, enquanto a Exposição ao Alagamento (EXP_ALAG) diminuiu 48.57%. Estes resultados quantificam a forte relação positiva entre as condições antrópicas e os riscos, validando a hipótese de que investimentos em infraestrutura (representados pela diminuição de CON_ANT) resultam em uma redução direta e mensurável dos riscos.